In [1]:
import re
import csv
import pandas as pd

from afinn import Afinn
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner'])

/hpc/applications/anaconda/3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
# add the preprocessing script: https://github.com/stephbuon/democracy-lab/issues/75
# create a copy of these property words and use them: https://docs.google.com/spreadsheets/d/1J1UofUSAeaOM0eWo8_1FbwgFHcris2SGZLH4XXXk21A/edit#gid=2100970163
# but I actually need to redo this list, and make a note, bc if I am using re.match then it will match with the start of the word onwards (not str contains style)

In [19]:
def list_contains(ls, keywords_list):
    filtered_list = []
    
    for string in ls:
        for keyword in keywords_list:
            if keyword in string:
                filtered_list.append(string)
                
    return filtered_list


def import_data(data, sep, col_name):
    hansard = pd.read_csv(data, sep=sep, usecols=[col_name])
    debate_text = hansard[col_name].tolist()
    return debate_text


def grammatical_collocates(ls, keywords_list, **kwargs):
    return_type = kwargs.get('return_type', None)
    
    if type(keywords_list) != list:
        raise TypeError('keywords_list must be a list.')
        
    regex = re.compile('|'.join(keywords_list))
    
    collocates = []
    
    for string in ls:
        doc = nlp(string)
        
        for token in doc:
            if regex.match(token.text):
                col = str(token.text) + ' ' + str(token.head.text)
                collocates.append(str(token.text) + ' ' + str(token.head.text))
                
                for child in token.children:
                    collocates.append(str(token.text) + ' ' + str(child))
                    
    if return_type == 'ls':
        return collocates
    if return_type == 'df':
        return pd.DataFrame(collocates, columns =['grammatical_collocates'])
    
    
def afinn_sentiment(text):
    return Afinn().score(text)


def textblob_sentiment(text):
    return TextBlob(text).sentiment.polarity


def vader_sentiment(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)


def sentiment_score(df, col_name):
    df['afinn'] = df[col_name].apply(afinn_sentiment)
    df['textblob'] = df[col_name].apply(textblob_sentiment)
    df['vader'] = df[col_name].apply(vader_sentiment)
    df['vader'] = df['vader'].apply(lambda score_dict: score_dict['compound'])    
    return df

def m(fname):
    data = import_data(fname, ',', 'text')
    
    # import keywords_list
    
    data = list_contains(data, keywords_list)
    
    # for keyword in property_keywords:
    
    collocates = grammatical_collocates(data, keyword, return_type='df')
    
    collocates_and_sentiment = sentiment_score(collocates, 'grammatical_collocates')
    
    collocates_and_sentiemnt.to_csv('collocates_sentiment_scores.csv', index=False)

In [9]:
out = import_data('/users/sbuongiorno/hansard_justnine_w_year.csv', ',', 'text')

In [10]:
test = out[:15]

In [11]:
test

['moved that Lord Walsingham be appointed chairman of the committee of privileges for the present session.',
 'in seconding the motion, took an opportunity to pay an handsome compliment to the talents and assiduity which the noble lord, who was the subject of the motion, had displayed during so long a period, in his official capacity of chairman of their lordships commit tees.',
 '—The question was put, and the appointment forthwith ordered by their lordships, nem. diss.',
 'then rose, and observed, that he was not in the habit of trespassing upon the attention, or taking up the valuable time of their lordships; but, after what had just fallen from the noble and learned lord on the woolsack, he could not avoid saying a word or two on the occasion.',
 "He had to return his thanks to the House for this additional manifestation of their lordships favourable opinion of his official conduct; he trusted he should persevere in that line of conduct which had met their lordships' approbation, w

In [12]:
out = list_contains(test, ['respect', ' it '])

In [20]:
save = grammatical_collocates(out, 'he', return_type='df')

TypeError: keywords_list must be a list.

In [16]:
save

,grammatical_collocates
0,except appointed
1,except in
2,his lordship
3,his intention
4,house of
5,house the
6,"house ,"
7,engaged was
8,engaged while
9,engaged actually


In [33]:
test = sentiment_score(save, 'grammatical_collocates')

In [34]:
test

,grammatical_collocates,afinn,textblob,vader
0,he begged,0.0,0.0,0.0
1,he begged,0.0,0.0,0.0
